In [2]:
import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import xlwt

In [3]:
files = "C:/Users/zhongj/Desktop/Task3_Data" #go to files direction
os.chdir(files)

### Sites Information

In [4]:
df0 = pd.read_excel("NYSERDA_DER_Metric_Data.xlsx", sheet_name = 'Facilities', 
                    usecols = ['Facility ID', 'Facility Name', 'ZIP', 'Latitude (°N)', 'Longitude (°E)', 
                               'Total Rated Electric Generation (kW)', 'Electric Utility', 'NYISO Zone'])
df0.columns =  ['Facility_ID', 'Facility_Name', 'ZIP', 'Latitude', 'Longitude', 
                'Total_Capacity', 'Electric_Utility', 'NYISO_Zone']

df1 = pd.read_excel("NYSERDA_DER_Metric_Data.xlsx", sheet_name = 'Projects',
                    usecols = ['Project ID', 'Facility ID', 'Technology'])
df1.columns = ['Project_ID', 'Facility_ID', 'Technology']

df2 = pd.read_excel("NYSERDA_DER_Metric_Data.xlsx", sheet_name = 'DE Resources',
                    usecols = ['Project ID', 'Tilt (° from horizontal)', 'Azimuth (° from North)'])
df2.columns =  ['Project_ID', 'Tilt', 'Azimuth']

In [5]:
df = pd.merge(df0, df1, on = ['Facility_ID'])

df = pd.merge(df, df2, on = ['Project_ID'])

df = df[df['Technology'] == 'Solar PV']
df = df.reset_index(drop = True)
del df['Technology']

df =  df[['Project_ID', 'Facility_ID', 'Facility_Name','Total_Capacity', 
               'ZIP', 'Latitude', 'Longitude', 'Tilt', 'Azimuth','NYISO_Zone', 'Electric_Utility']]

In [ ]:
writer = pd.ExcelWriter(r'Sites_Information.xlsx', 
                                engine = 'xlsxwriter')
df.to_excel(writer, index= False)

writer.save()

### Solar Data for sites

In [ ]:
#Here using Solar_2019_01_through_2021_03_01_first_edit file

# I editted the original csv file manually using notepad, which I deleted the second line (this line contains names of sites)

import csv
file_path = "Solar_2019_01_through_2021_03_01_first_edit.csv" #input file 
file_path2 = "All_Sites_BTM_Solar_Data_Edit_15_Min_Interval.csv" #output file

with open(file_path, 'r', newline='') as inputfile: #read file
    with open(file_path2, 'w', newline='') as outputfile: 
        reader = csv.reader(inputfile) 
        writer = csv.writer(outputfile)
        for row in reader:
            new_row = ['' if ('-' in i) else i for i in row] #changing all the '-' to Nothing
            writer.writerow(new_row) 
            
inputfile.close()
outputfile.close()

In [ ]:
df3= pd.read_csv("All_Sites_BTM_Solar_Data_Edit_15_Min_Interval.csv") 

In [ ]:
df4 = df3.iloc[:, 1:].astype(float) #convert all the data to float

Project_ID = df4.columns #Get all the sites project id

In [ ]:
#Get times
df4['DateTime'] = pd.to_datetime(df3['date time'])
df4["Date"] = df4['DateTime'].dt.date
df4["Year"] = df4['DateTime'].dt.year
df4["Month"] = df4['DateTime'].dt.month
df4["Day"] = df4['DateTime'].dt.day
df4["Hour"] = df4['DateTime'].dt.hour
df4["Min"] = df4['DateTime'].dt.minute

In [ ]:
date = list(df4['Date'].unique()) #get all unique dates
year = list(df4['Year'].unique()) #get all unique years

In [ ]:
# Change Hours since the timestamp represntative interval ending
df4['Hour'] = df4.apply(lambda x:(x['Hour']-1) if (x['Min']==0) else(x['Hour']), axis=1)
df4['Hour'] = df4.apply(lambda x:(x['Hour']+1) , axis=1)
df4 = df4[df4['Hour']<24]

In [ ]:
# for any day, if there is no solar generation behaviors pattern from 6am to 7pm in that dat, then change all data in that day to None
# rule:
    # if there are more that 24 data(15 mins interval) from 7am to 6pm in that day has the same values(values greater that 0), then it has  no solar generation behaviors pattern
    #(7am to 6pm is 12 hours with 15 mins internval in each hour, so there will be 48 data)


from collections import Counter

list_date = []
for p in Project_ID:
    for d in date:
        #choose data: anyday, from  7am to 6pm, values greater that 0
        df_s = df4[(df4['Date']==d)&(df4[p]>0)&(df4['Hour']>6)&(df4['Hour']<19)].reset_index() 
        s = list(df_s[p]) #convert data to list
        
        if len(s)>0: #if there is data
            
            # using Counter function in Python3, get a dictionary
            # The dictionary contains each number(solar generation) and its frequency
            count = dict(Counter(s))  
            max_value  = max(count.values())
            
            if max_value > 24: #if it shows up more than 24 times
                 df4[p].loc[df4['Date']==d] = None

In [ ]:
# Change the columns position:

c = ['DateTime', 'Year','Month', 'Day', 'Hour','Min']
for p in Project_ID:
    c.append(p)
df4 =  df4[c]

#### delete all the solar values that are too much larger than the capacity

In [6]:
df4= pd.read_csv("All_Sites_BTM_Solar_Data_Edit_15_Min_Interval.csv")

In [15]:
df5 = df4.groupby(['Year', 'Month', 'Day','Hour']).sum()
df5 = df5.reset_index()
del df5['Min']

In [16]:
Project_ID = list(df4.columns)
Project_ID.remove('DateTime')
Project_ID.remove('Year')
Project_ID.remove('Month')
Project_ID.remove('Day')
Project_ID.remove('Hour')
Project_ID.remove('Min')

In [17]:
dff = df.copy()
dff.index = dff['Project_ID']
sites2 = list(map(int, Project_ID))
dff = dff.loc[sites2,:]
dff = dff.reset_index(drop = True)

In [18]:
for p in Project_ID:
    total_C = dff[dff['Project_ID']== int(p)]['Total_Capacity'].values[0]
    df5[p] = df5[p]/total_C

In [ ]:
for s in Project_ID:
    df_s = df5[['Year', 'Month', 'Day', 'Hour',s]]
    for index, row in df_s.iterrows():
        if (row[s]> 5):
            print(df4[(df4['Year']==row['Year'])&(df4['Month']==row['Month'])&(df4['Day']==row['Day'])&(df4['Hour']==row['Hour'])][s])

In [14]:
df4.to_csv(r'All_Sites_BTM_Solar_Data_Edit_15_Min_Interval.csv', index = False)

In [22]:
df5 = df4.groupby(['Year', 'Month', 'Day','Hour']).sum()
df5 = df5.reset_index()
del df5['Min']